**ÉTAPE 0** : préparation des données

In [2]:
import process 
import pandas as pd
import numpy as np

# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
# Jerome path : 'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DATA_Statapp.csv'
# Mohamed path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
df = pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')

# Apply conversion function to 'open' and 'close' columns
df['open'] = df['open'].apply(process.safe_literal_eval)
df['close'] = df['close'].apply(process.safe_literal_eval)

# Calculate returns for each line
df['return'] = df.apply(lambda row: [(close - open) / open for open, close in zip(row['open'], row['close'])], axis=1)

new_df = df[['ticker', 'return']] # create a new data frame with the column ticker and return 

# Créons le DataFrame à partir des listes dans 'return'
# On suppose ici que 'new_df' est déjà défini et contient la colonne 'return'

# Convertir chaque liste dans la colonne 'return' en plusieurs colonnes dans le nouveau DataFrame
returns_df = pd.DataFrame(new_df['return'].tolist())

# Ajouter la colonne 'ticker' du 'new_df' au début de 'returns_df'
returns_df.insert(0, 'ticker', new_df['ticker'])

# Renommer les colonnes pour refléter qu'elles sont des rendements
returns_df.columns = ['ticker'] + [f'return_{i}' for i in range(len(returns_df.columns) - 1)]

df_cleaned = process.remove_rows_with_nan(returns_df)
df_cleaned.reset_index(drop=True, inplace=True)

process.check_nan_inf(df_cleaned)

df_cleaned.set_index('ticker', inplace=True) ## ces deux lignes sont fondamentales, ne pas les retirer !!
df.set_index('ticker', inplace=True)


There are no NaN values in the dataframe


In [2]:
import process 
lookback_window = [5000, 5250]
corr_matrix = process.correlation_matrix(lookback_window=lookback_window, df_cleaned=df_cleaned)
cluster_composition_and_centroid = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=corr_matrix, number_of_clusters=20, lookback_window=lookback_window)
constituent_weights = process.constituent_weights(
    df_cleaned=df_cleaned, 
    cluster_composition=cluster_composition_and_centroid, 
    sigma=10, 
    lookback_window=lookback_window
)
cluster_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, df=df, lookback_window=lookback_window)

/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [3]:
cov_matrix = cluster_return.transpose().cov()
cluster_return_target = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, df=df, lookback_window=[lookback_window[1], lookback_window[1]+1])
expected_returns = process.noised_array(y=cluster_return_target, eta=0.01).iloc[:, 0].values.squeeze()

In [5]:
expected_returns

array([0.11109142622824658, 0.037086691083648045, 0.010868973412432871,
       -0.030390075230632578, 0.1104432443012554, -0.03229696978735891,
       0.006301345741715764, -0.04159951963632331, -0.07714246425765925,
       0.0074841397277071045, 0.010886827717384098, 0.012574136197149878,
       -0.00572551058761697, 0.0052557648145970335, 0.04458364426966589,
       0.008451062720864239, 0.0557381394353541, -0.042330814954758364,
       0.02869986818254125], dtype=object)

In [19]:
from pypfopt.efficient_frontier import EfficientFrontier

ef = EfficientFrontier(expected_returns=expected_returns, cov_matrix=cov_matrix, weight_bounds=(0, 1))
ef. efficient_return(target_return=expected_returns.mean())


/usr/local/lib/python3.11/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


1.0

In [4]:
lookback_window = [0, 250]
consolidated_W = process.consolidated_W(number_of_repetitions=5, lookback_window=lookback_window, df_cleaned=df_cleaned, number_of_clusters=20, sigma=10e-2, df=df, eta=0.01)

/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, spe

In [5]:
portfolio_return = process.portfolio_returns(evaluation_window=10, df_cleaned=df_cleaned, lookback_window=lookback_window, consolidated_W=consolidated_W, df=df)

In [7]:
portfolio_return.cumsum()

,portfolio return
return_250,0.004725
return_251,0.003910
return_252,0.010020
return_253,0.013736
return_254,0.013734
return_255,0.014510
return_256,0.020217
return_257,0.022750
return_258,0.025810
return_259,0.025914
